# Chapter 8 - LLM Agents Development Ecosystem

In this chapter we will leverage crewAI and Ollama to demonstrate how to create multiple agents collborating with each other
to perform a given task.

- ollama
- llamaindex
- langchain
- deeplake

## Ollama

Ollama enables us to run open source LLMs locally. A tool that enables local deployment of large language models. A great tool for experimetning with LLMs. No need cloud hosting.
http://github.com/ollama/ollama. Refer to the website for installation.

Advantages for using Ollama,

1. Quick iteratvie development without needing to deploy model changes
2. Privacy and security - data does not leave your machine.
3. cost - more cost effective than making API calls.
4. control - more control over the model

    curl  -fsSL https://ollama.com/install.sh | sh

    >>> Installing ollama to /usr/local/bin...
    >>> Creating ollama user...
    >>> Adding ollama user to render group...
    >>> Adding ollama user to video group...
    >>> Adding current user to ollama group...
    >>> Creating ollama systemd service...
    >>> Enabling and starting ollama service...
    Created symlink /etc/systemd/system/default.target.wants/ollama.service → /etc/systemd/system/ollama.service.
    >>> NVIDIA GPU installed.

    sudo systemctl status ollama
    ● ollama.service - Ollama Service
         Loaded: loaded (/etc/systemd/system/ollama.service; enabled; vendor preset: enabled)
         Active: active (running) since Sat 2024-05-11 19:59:42 EDT; 5min ago
       Main PID: 1053781 (ollama)
          Tasks: 18 (limit: 18827)
         Memory: 480.5M
         CGroup: /system.slice/ollama.service
                 └─1053781 /usr/local/bin/ollama serve
    
    May 11 19:59:42 gopi-G5-MD ollama[1053781]: Couldn't find '/usr/share/ollama/.ollama/id_ed25519'. Generating new private key.
    May 11 19:59:42 gopi-G5-MD ollama[1053781]: Your new public key is:
    May 11 19:59:42 gopi-G5-MD ollama[1053781]: ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIPkwrlZDwOjnkF2xAiAWVxs8CrIfsqSnNcs3adQQv9xC
    May 11 19:59:42 gopi-G5-MD ollama[1053781]: 2024/05/11 19:59:42 routes.go:1006: INFO server config env="map[OLLAMA_DEBUG:false OLLAMA_LLM_LIBRARY: OLLAMA_MAX_LOADED_MODELS:1 OLLAMA_MAX_QUEUE:512 OLLAMA_>
    May 11 19:59:42 gopi-G5-MD ollama[1053781]: time=2024-05-11T19:59:42.753-04:00 level=INFO source=images.go:704 msg="total blobs: 0"
    May 11 19:59:42 gopi-G5-MD ollama[1053781]: time=2024-05-11T19:59:42.753-04:00 level=INFO source=images.go:711 msg="total unused blobs removed: 0"
    May 11 19:59:42 gopi-G5-MD ollama[1053781]: time=2024-05-11T19:59:42.753-04:00 level=INFO source=routes.go:1052 msg="Listening on 127.0.0.1:11434 (version 0.1.36)"
    May 11 19:59:42 gopi-G5-MD ollama[1053781]: time=2024-05-11T19:59:42.753-04:00 level=INFO source=payload.go:30 msg="extracting embedded files" dir=/tmp/ollama769771873/runners
    May 11 19:59:44 gopi-G5-MD ollama[1053781]: time=2024-05-11T19:59:44.699-04:00 level=INFO source=payload.go:44 msg="Dynamic LLM libraries [cpu cpu_avx cpu_avx2 cuda_v11 rocm_v60002]"
    May 11 19:59:45 gopi-G5-MD ollama[1053781]: time=2024-05-11T19:59:45.929-04:00 level=INFO source=types.go:71 msg="inference compute" id=0 library=cpu compute="" driver=0.0 name="" total="15.4 GiB" avail>
    lines 1-19/19 (END)




# List of available models

    ollama list
    NAME	ID	SIZE	MODIFIED 

# Pull a model

     ollama pull phi3
    pulling manifest 
    pulling 4fed7364ee3e... 100% ▕███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏ 2.3 GB                         
    pulling c608dc615584... 100% ▕███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏  149 B                         
    pulling fa8235e5b48f... 100% ▕███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏ 1.1 KB                         
    pulling d47ab88b61ba... 100% ▕███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏  140 B                         
    pulling f7eda1da5a81... 100% ▕███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏  485 B                         
    verifying sha256 digest 
    writing manifest 
    removing any unused layers 
    success 


    (base) gopi@gopi-G5-MD:~/Documents/small_llm$ ollama list
    NAME       	ID          	SIZE  	MODIFIED       
    phi3:latest	a2c89ceaed85	2.3 GB	19 seconds ago	


Now listing the model shows phi3 model. The models are stored as blob in /usr/share/ollama/.ollama

Ollama bundles the model into a single package defined by a Modelfile. Let us see the modelfile for phi3.

    >>> /show modelfile
    # Modelfile generate by "ollama show"
    # To build a new Modelfile based on this, replace FROM with:
    # FROM phi3:latest
    
    FROM /usr/share/ollama/.ollama/models/blobs/sha256-4fed7364ee3e0c7cb4fe0880148bfdfcd1b630981efa0802a6b62ee52e7da97e
    TEMPLATE "{{ if .System }}<|system|>
    {{ .System }}<|end|>
    {{ end }}{{ if .Prompt }}<|user|>
    {{ .Prompt }}<|end|>
    {{ end }}<|assistant|>
    {{ .Response }}<|end|>
    "
    PARAMETER stop <|user|>
    PARAMETER stop <|assistant|>
    PARAMETER stop <|system|>
    PARAMETER stop <|end|>
    PARAMETER stop <|endoftext|>
    PARAMETER num_keep 4
    LICENSE """Microsoft.
    Copyright (c) Microsoft Corporation.
    
    MIT License
    
    Permission is hereby granted, free of charge, to any person obtaining a copy
    of this software and associated documentation files (the "Software"), to deal
    in the Software without restriction, including without limitation the rights
    to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
    copies of the Software, and to permit persons to whom the Software is
    furnished to do so, subject to the following conditions:
    
    The above copyright notice and this permission notice shall be included in all
    copies or substantial portions of the Software.
    
    THE SOFTWARE IS PROVIDED *AS IS*, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
    IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
    FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
    AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
    LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
    OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
    SOFTWARE."""


In [ ]:
## Interacting with LLM

    ollama run phi3:latest
    >>> Why is the sky blue in color?
     The sky appears blue to our eyes because of a phenomenon called Rayleigh scattering. As sunlight travels through Earth's atmosphere, it encounters molecules and small particles that are much 
    smaller than its wavelength. These particles scatter shorter-wavelength light (blue and violet) more effectively than longer-wavelength light (red, orange).
    
    However, our eyes are less sensitive to violet light, and the Sun emits less violet light compared to blue light; that's why we see a predominantly blue sky during daytime. At sunrise or sunset, 
    when the light path through the atmosphere is longer, even more scattering of red and orange wavelengths occurs, which makes the sky appear in shades of red, pink, and orange.
    
    The overall color perception can also be affected by atmospheric conditions such as pollution or dust particles. But under normal circumstances on a clear day, Rayleigh scattering is responsible 
    for the blue coloration of our sky.
    
    >>> Send a message (/? for help)



In [ ]:
## Prompt template



### Interacting with Langchain

```{figure} ../images/chapter5/LangChain.png
---
height: 200px
name: directive-fig
---
Lanchain Ecosystem
-----

A framework for building large language models powered application.


The core components of Langchain are

* Model I/O
    Building blocks to interface with any language model
* Retrieval
* Composition

## Model I/O

In [39]:
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama

MODEL = "phi3:latest"

llm = Ollama(model=MODEL)
chat_model = ChatOllama(model=MODEL)

In [55]:
out = llm.invoke("Explain anitbiotics")

print(out)

Antibiotics are powerful medicines that fight bacterial infections. They work by either killing the bacteria or slowing down their growth, helping your immune system to eliminate the infection more effectively. Antibiotics do not help against viral infections such as colds and flu.

Here's a brief overview of how antibiotics work:

1. **Target Bacterial Cells**: Antibiotics are designed to target specific structures or processes within bacteria, which human cells do not possess. This selectivity helps minimize harm to the host organism while effectively attacking the pathogens.
   
2. **Killing/Inhibiting Growth**: There are two main types of antibiotics – Bactericidal (kills bacteria) and Bacteriostatic (inhibits growth). Bactericidal antibiotics, like penicillin, directly destroy bacterial cells. Bacteriostatic antibiotics, such as tetracycline, inhibit the ability of bacteria to grow or reproduce.
   
3. **Different Modes of Action**: Antibiotics can work through various mechanisms,

In [56]:
out = llm.invoke("Explain anitbiotics A:")
print(out)

Antibiotics are powerful medications that fight bacterial infections by either killing the bacteria or inhibiting their growth. They work through various mechanisms, and here's a basic overview of antibiotics:

1. Classes of Antibiotics: There are several classes of antibiotics, each with unique properties and functions. Some common classes include:
   - Penicillins (e.g., amoxicillin): Inhibit bacterial cell wall synthesis; first-line treatment for various infections.
   - Cephalosporins (e.g., ceftriaxone): Similar to penicillins, but with broader coverage against resistant bacteria.
   - Macrolides (e.g., azithromycin and erythromycin): Inhibit protein synthesis by binding to the 50S ribosomal subunit in bacteria; used for respiratory, skin, or sexually transmitted infections.
   - Tetracyclines (e.g., doxycycline and minocycline): Also target protein synthesis by binding to the 30S ribosomal subunit, commonly prescribed for a wide range of bacterial infections.
   - Aminoglycosides

In [57]:
query = out + "\n\n summarize it in five sentences."
out = llm.invoke(query)
print(out)

Antibiotics are medications used to treat bacterial infections by either killing the bacteria or hindering their growth, with various classes targeting different aspects of bacterial cellular functions. These classes include penicillins, cephalosporins, macrolides, tetracyclines, aminoglycosides, fluoroquinolones, glycopeptides, and sulfonamides, each with specific mechanisms of action such as inhibiting cell wall synthesis or disrupting protein production. It's crucial to use antibiotics judiciously due to the rise of bacterial resistance, necessitating appropriate prescription selection and adherence to dosage guidelines by healthcare professionals. Patients must also complete their antibiotic course as directed to minimize side effects like nausea or severe allergic reactions while avoiding conditions such as Clostridium difficile infection. Balancing the therapeutic benefits of antibiotics against potential risks is essential for effective treatment and preserving public health.


In [52]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("what would be a {adjective} company name for a sells {product}?")
prompt = prompt_template.format(adjective="good", product="computer vision analytics")
parser = StrOutputParser()
chain = prompt | llm | paraser
chain.invoke()

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'str'>

In [49]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage(content=text)]
chat_model.invoke(messages)

AIMessage(content="1. VisionAnalytica: A blend of 'vision' and 'analytics,' suggesting expertise in using computer vision to analyze data effectively.\n\n2. InsightVista: Combining the idea of insight (understanding) with a vista (view), indicating an expansive approach to insights through computer vision analytics.\n\n3. PixelPulse Analytics: Merging 'pixel' from image processing and 'pulse' to indicate action-oriented, real-time data analysis using computer vision technology.\n\n4. EyeDataTech: Playing on the idea of focusing ('eye') on data through cutting-edge technology (tech).\n\n5. DeepVision Analytics: A nod to deep learning and neural networks used in computer vision, combined with 'analytics' for their role in interpreting complex visual data.\n\n6. OptiFrame Insights: Fusing the concepts of optimization ('opti') and structural elements ('frame'), implying precision-driven insights from frame-by-frame analysis.\nean to convey a blend of technological innovation and insightful

## Composition

### Tools

Provide an interface for LLMs and otehr components to interact with other sytems.

1. Name of the tool
2. a description of what the tool is
3. JSON schema of what the inputs to the tool are
4. The function to call
5. whether the result of a tool should be returned directly to the user.

name, description and json schema can be used to prompt the LLM. Based on this the LLM decides what action to take. The function calling can perform the action.



In [16]:
# Built in tool example
# pre-reqs
#      pip3 install wikipedia


from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)



In [22]:
print(f"name: {tool.name}")
print(f"desc: {tool.description}")
print(f"args: {tool.args}")

name: wikipedia
desc: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
args: {'query': {'title': 'Query', 'type': 'string'}}


In [23]:
query = {"query": "langchain"}
tool.run(query)

'Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications '

In [ ]:
### Built in tools



In [27]:
from langchain.tools import DuckDuckGoSearchRun, DuckDuckGoSearchResults

search_tool = DuckDuckGoSearchResults()
search_tool.run("langchain builtin tools")

'[snippet: from langchain.tools import tool @tool def add(a: int, b: int) -> int: """Adds two numbers together""" # this docstring gets used as the description return a + b # the actions our tool performs. Much simpler right. Behind the scenes, the decorator magically uses the method provided to extend the BaseTool class, just as we did earlier., title: Building a simple Agent with Tools and Toolkits in LangChain, link: https://towardsdatascience.com/building-a-simple-agent-with-tools-and-toolkits-in-langchain-77e0f9bd1fa5], [snippet: To understand how LangChain agents work, let\'s first create a custom agent. In a later section, we\'ll will explain how you can use built-in LangChain agents. Creating Tools. LangChain agents use tools to interact with third-party applications. Depending on the User prompt, an agent can use one or multiple tools to perform a task., title: Introduction to Custom and Built-in Python LangChain Agents, link: https://wellsr.com/python/working-with-python-lang

### Toolkits

a collection of tools that are designed to be used together for specific tasks. They have convenient loading methods.

In [ ]:
### Retrievers

A retriever is an interface that returns documents given an unstructured query.

In [33]:
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader



In [30]:
from langchain.retrievers.web_research import WebResearchRetriever



In [6]:
from langchain_community.llms import Ollama

llm = Ollama(model="gemma:2b-instruct")

llm.invoke("Tell me a joke")

'Why did the math book look so sad?\n\nBecause of all of its problems!'

## Agents

<Need to rewrite this one>
Of course, many studies show that they have a generalization ability, where they might work well even on questions or tasks that don’t appear in training data. However, even if you have a really well-trained, the model itself is limited by their training data. What I mean is that you can not provide enough knowledge for them in some cases. The recent LLMs can do math pretty well, but they are not built for calculating tasks. Therefore, they will struggle with complex math equations.

Another example is the limitation of common knowledge. You take years for collecting data to train an LLM. But if you ask your LLM about the result of the game between Arsenal and Brighton yesterday, they just simply can’t answer correctly. The model can’t access the internet to find out, and the training data doesn’t have that information. That’s the reason we should connect LLMs with other tools to expand their power. And the agent is the one we want.

An agent is a program that can “think” and interact with “the world”. Given a task, it will think about how to solve it, make a plan, do some actions to the environment, receive feedback, and repeat these steps until the task is done. For example, we can see a cleaning robot as an agent. We assign it the task of cleaning the bedroom. The robot received the requirement from the human. Then, it starts to think and make a plan for cleaning. It does some actions: go to the bedroom, start to collect garbage, etc. The sensors may help it to check its location, and avoid hitting stuff. The cleaning robot can interact with and receive feedback from the environment.

An LLM agent is also the same. In this case, we exploit the LLM to help the agent can “think” since LLMs are quite good at reasoning tasks. To interact with “the world”, we add some tools that can act and receive results from “the world”.

In [2]:
llm

Ollama(model='gemma:2b')

In [7]:
from langchain_core.tools import tool


@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int

In [8]:
llm_with_tools = llm.bind_tools([multiply])

AttributeError: 'Ollama' object has no attribute 'bind_tools'

In [ ]:
https://medium.com/data-science-at-microsoft/harnessing-the-power-of-large-language-models-a-comparative-overview-of-langchain-semantic-c21f5c19f93e

Difference between semantic kernel, autogen, langchain.

### Function Calling

In [10]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions

ModuleNotFoundError: No module named 'langchain_experimental'

## Chroma

In [59]:
import chromadb

chroma_client = chromadb.Client()
collections = chroma_client.create_collection(name="my_collection")

collections.add(
    documents=[
        "This is a document about pineapple"
       ,"This is a document about oranges"
    ],
    ids=["id1","id2"]
)

/home/gopi/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 79.3M/79.3M [00:10<00:00, 7.75MiB/s]


In [60]:
results = collections.query(
    query_texts=["This is a query document about hawaii"],
    n_results=2
)

print(results)

{'ids': [['id1', 'id2']], 'distances': [[1.0404009819030762, 1.2430799007415771]], 'metadatas': [[None, None]], 'embeddings': None, 'documents': [['This is a document about pineapple', 'This is a document about oranges']], 'uris': None, 'data': None}


## LlamaIndex

When you single out an LLM for your usecase, you understand the though the LLM is pretrained on a large amount of publicly available dataset, it still has no clue about your domain/use case. You need to make your data available to the LLM for it to perform meaningfully. Context Augmentation is the name of the technique with which you provide your data to the LLM for it to perform inference for your use case.

Llamaindex provides a framework for context augmentation. It provides the following tools to make your data augmentation task easier,

1. Data connectors to ingest data from a variety of sources including pdf, APIs, SQL and many others.
2. Data indexes to structure the ingested data in a vector format.
3. Natural language access to our data through Engines.
4. Knowledge works in the form of Agents
5. Tools to perform experiments to enable observability and evaluation of your application performance.

Llamaindex started with a framework for connecting to different datasources, but now have evolved into a complete ecosystem for building RAG based agents and other LLM based applications.

In [9]:
!ls ../data/chapter7

old-man-and-the-sea.txt


### Data connector

The class Readers implements the data connectors.The open source project LlamaHub hosts different types of data connectors to support multiple dataformats.



In [37]:
import urllib.request
import os
from pathlib import Path

url = ("https://raw.githubusercontent.com/Zhenye-Na/data-structures-uiuc/"
       "master/lab_huffman/data/old-man-and-the-sea.txt"
      )
filename = url.split('/')[-1]

parent = Path(os.getcwd()).parent.absolute()
path   = Path(parent, "data/chapter7/", filename)

urllib.request.urlretrieve(url, path)

(PosixPath('/home/gopi/Documents/small_llm/llmbook/data/chapter7/old-man-and-the-sea.txt'),
 <http.client.HTTPMessage at 0x75fdeb50a1d0>)

In [38]:
from llama_index.core import SimpleDirectoryReader

oldmansea = SimpleDirectoryReader(path.parent.absolute()).load_data()

In [47]:
for document in oldmansea:
    print(f"Document id:\n {document.doc_id} \n\n metadata:\n {document.metadata}")


Document id:
 b62f21f0-b657-4559-b47b-a44040f0923c 

 metadata:
 {'file_path': '/home/gopi/Documents/small_llm/llmbook/data/chapter7/old-man-and-the-sea.txt', 'file_name': 'old-man-and-the-sea.txt', 'file_type': 'text/plain', 'file_size': 43188, 'creation_date': '2024-08-09', 'last_modified_date': '2024-08-09'}


Document is a generic container for any data source. They can be constructed manually or through data loaders. The previous example we used SimpleDirectoryReader to create a document container for old-man-and-sea.txt file in the local directory. In addition to data Document also stores the metadata.

## Nodes

For indexing purpose, the Document is broken down into chunks. A Node represents a chunk.



In [51]:
from llama_index.core.node_parser import SentenceSplitter

parser = SentenceSplitter()
nodes = parser.get_nodes_from_documents(oldmansea)

In [56]:
len(nodes)

14

In [54]:
for node in nodes:
    print(nod

The Project Gutenberg EBook of The Old Man of the Sea, by W.W. Jacobs

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.net


Title: The Old Man of the Sea
       Ship's Company, Part 11.

Author: W.W. Jacobs

Release Date: January 1, 2004 [EBook #10571]

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK THE OLD MAN OF THE SEA ***




Produced by David Widger




SHIP'S COMPANY

By W.W. Jacobs



THE OLD MAN OF THE SEA


"What I want you to do," said Mr. George Wright, as he leaned towards the
old sailor, "is to be an uncle to me."

"Aye, aye," said the mystified Mr. Kemp, pausing with a mug of beer
midway to his lips.

"A rich uncle," continued the young man, lowering his voice to prevent
any keen ears in the next bar from acquiring useless knowledge.  "An
uncle from New Zealand, 

In [49]:
from llama_index.core.schema import MetadataMode

for document in oldmansea:
    print(f"What LLM sees:\n{document.get_content(MetadataMode.LLM)}")

What LLM sees:
file_path: /home/gopi/Documents/small_llm/llmbook/data/chapter7/old-man-and-the-sea.txt

The Project Gutenberg EBook of The Old Man of the Sea, by W.W. Jacobs

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.net


Title: The Old Man of the Sea
       Ship's Company, Part 11.

Author: W.W. Jacobs

Release Date: January 1, 2004 [EBook #10571]

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK THE OLD MAN OF THE SEA ***




Produced by David Widger




SHIP'S COMPANY

By W.W. Jacobs



THE OLD MAN OF THE SEA


"What I want you to do," said Mr. George Wright, as he leaned towards the
old sailor, "is to be an uncle to me."

"Aye, aye," said the mystified Mr. Kemp, pausing with a mug of beer
midway to his lips.

"A rich uncle," continued the young man, lowering his voice 

### Natural Language SQL Queries

In [80]:
from sqlalchemy import (
    create_engine
   ,MetaData
   ,Table
   ,Column
   ,String
   ,Integer
   ,select
)


engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

table_name = "cricket_batsmen_stats"

batsment_stats_table = Table(
    table_name
   ,metadata_obj
   ,Column("name", String(50), primary_key=True)
   ,Column("matches", Integer)
   ,Column("runs", Integer)
)

metadata_obj.create_all(engine)



In [81]:
from llama_index.readers.web import SimpleWebPageReader
import pandas as pd
from bs4 import BeautifulSoup


documents = SimpleWebPageReader(html_to_text=False).load_data(
    ["https://www.espncricinfo.com/records/most-runs-in-career-83548"]
)
html_page = documents[0].text

soup = BeautifulSoup(html_page)
tables = pd.read_html(str(soup))

players_table = tables[0]
players_table.head()

,Player,Span,Mat,Inns,NO,Runs,HS,Ave,BF,SR,100,50,0,4s,6s
0,SR Tendulkar (IND),1989-2012,463,452,41,18426,200*,44.83,21368,86.23,49,96,20,2016,195
1,KC Sangakkara (Asia/ICC/SL),2000-2015,404,380,41,14234,169,41.98,18048,78.86,25,93,15,1385,88
2,V Kohli (IND),2008-2024,295,283,44,13906,183,58.18,14866,93.54,50,72,16,1302,151
3,RT Ponting (AUS/ICC),1995-2012,375,365,39,13704,164,42.03,17046,80.39,30,82,20,1231,162
4,ST Jayasuriya (Asia/SL),1989-2011,445,433,18,13430,189,32.36,14725,91.20,28,68,34,1500,270


In [82]:
players_table = players_table[['Player','Mat','Runs']]
players_table.columns = ['name','matches','runs']
players_table.head()

,name,matches,runs
0,SR Tendulkar (IND),463,18426
1,KC Sangakkara (Asia/ICC/SL),404,14234
2,V Kohli (IND),295,13906
3,RT Ponting (AUS/ICC),375,13704
4,ST Jayasuriya (Asia/SL),445,13430


In [83]:
insert_records = players_table.to_dict(orient='records')

In [86]:
from sqlalchemy import insert

for record in insert_records:
    stmt = insert(batsment_stats_table).values(**record)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

In [87]:
from sqlalchemy import text

with engine.connect() as con:
    rows = con.execute(text(f"select name from {table_name} limit 5"))

for row in rows:
    print(row)

('A Flower (ZIM)',)
('A Jadeja (IND)',)
('A Ranatunga (SL)',)
('A Symonds (AUS)',)
('AB de Villiers (Afr/SA)',)


In [91]:
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.core import SQLDatabase

sql_database = SQLDatabase(engine, include_tables=[table_name])
query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database, tables=[table_name], llm=None
)

query_str = "which batsmen has the most runs?"
response = query_engine.query(query_str)

ValueError: 
******
Could not load OpenAI model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

To disable the LLM entirely, set llm=None.
******